In [1]:
# ============= RubyGems and Bundler preambles ==============

$".append("io/wait.rb", "socket.rb")
# HACK: trick bundler to think that we are supporting https
module OpenSSL
  module SSL
    VERIFY_PEER = 0
    class SSLError < StandardError; end
  end
end

class Socket
  class << self
    def method_missing(sym, *) = nil
  end
end

class SocketError; end

ENV["HOME"] = "/home/me"

class Thread
  def self.new(&block)
    f = Fiber.new(&block)
    def f.value = resume
    f
  end
end

def File.chmod(mode, *paths) = nil
class File
  def chmod(mode) = nil
end

require "js"
require "rubygems/commands/install_command"

class JS::Object
  def to_a
    Array.new(self[:length].to_i) {|i| self.call(:at, i).to_i }
  end
end

class Fetch
  def self.request(uri, request)
    # Use proxy for RubyGems server to enable CORS
    if uri.hostname == "rubygems.org" or uri.hostname == "index.rubygems.org"
      uri.hostname = "irb-wasm-proxy.edgecompute.app"
    end
    options = JS.eval("return {}")
    options[:method] = request.method.to_s
    options[:headers] = JS.eval("return {}")
    request.each_capitalized do |k, v|
      options[:headers][k] = v
    end
    response = JS.global.fetch(uri.to_s, options).await
    content_type = response[:headers].get("Content-Type").to_s
    is_octet_stream = content_type.start_with?("application/octet-stream")
    if uri.path.end_with?(".gem") or uri.path.end_with?(".rz") or is_octet_stream
      # FIXME: support class constructor and direct function call
      factory = JS.eval("return { make: (data) => new Uint8Array(data) }")
      body = factory.make(response.arrayBuffer.await).to_a.pack("C*")
    else
      body = response.text.await.to_s
    end
    body_str = Net::BufferedIO.new(StringIO.new body)

    status = response["status"].inspect
    response_class = Net::HTTPResponse::CODE_TO_OBJ[status]
    response = response_class.new("2.0", status.to_i, nil)
    response.reading_body(body_str, true) {}
    # response.instance_variable_set(:@read, false)
    response
  end
end
class Gem::Request
  def self.request(uri, request)
    response = Fetch.request uri, request
    response
  end

  def perform_request(request) = Gem::Request.request(@uri, request)
end

class Gem::Installer
  def build_extensions
      # HACK: skip ext build for now...
  end
end

class Gem::Specification
  # HACK: supress missing extension warning, which prevents "require" to work
  def missing_extensions? = false
end

def Gem.user_home = Dir.home
# HACK: Install gems under writable directory by default
def Gem.dir = Gem.user_dir

Gem.configuration.concurrent_downloads = 1

require "bundler"
class Bundler::ProcessLock
  def self.lock(*)
    # HACK: no flock on browser...
    yield
  end
end

class Bundler::Fetcher
  class WebConnection
    def request(uri, request) = Gem::Request.request(uri, request)
  end
  def connection
    @connection ||= WebConnection.new
  end
end

# HACK: mark OpenSSL::Digest unavailable
module Bundler::SharedHelpers
  def md5_available? = false
end

puts "RubyGems and Bundler are activated"

RubyGems and Bundler are activated


In [2]:
# HACK
$" << "enumerable/statistics/extension"; nil;

In [3]:
require "bundler/inline"

gemfile(true) do
    source 'https://rubygems.org'
    gem "charty"
    gem "red-datasets-daru"
end

Fetching gem metadata from https://rubygems.org/...............
Resolving dependencies...
Fetching backports 3.23.0
Installing backports 3.23.0
Using bundler 2.4.0.dev
Fetching pycall 1.4.1
Installing pycall 1.4.1 with native extensions
Fetching matplotlib 1.2.0
Installing matplotlib 1.2.0
Fetching numpy 0.4.0
Installing numpy 0.4.0
Fetching pandas 0.3.8
Installing pandas 0.3.8
Fetching concurrent-ruby 1.1.10
Installing concurrent-ruby 1.1.10
Fetching mime-types-data 3.2022.0105
Installing mime-types-data 3.2022.0105
Fetching mime-types 3.4.1
Installing mime-types 3.4.1
Fetching playwright-ruby-client 1.27.0
Installing playwright-ruby-client 1.27.0
Using matrix 0.4.2
Fetching red-colors 0.3.0
Installing red-colors 0.3.0
Fetching csv 3.2.5
Installing csv 3.2.5
Using rexml 3.2.5
Fetching rubyzip 2.3.2
Installing rubyzip 2.3.2
Fetching red-datasets 0.1.5
Installing red-datasets 0.1.5
Fetching red-palette 0.5.0
Installing red-palette 0.5.0
Fetching charty 0.2.12
Installing charty 0.2.12
Fe

[<Bundler::Dependency type=:runtime name="charty" requirements=">= 0">, <Bundler::Dependency type=:runtime name="red-datasets-daru" requirements=">= 0">]

In [8]:
class Datasets::Downloader
  def download(output_path)
    return if output_path.exist?

    output_path.parent.mkpath

    headers = {
      "Accept-Encoding" => "identity",
      "User-Agent" => "Red Datasets/#{Datasets::VERSION}",
    }
    start = nil
    partial_output_path = Pathname.new("#{output_path}.partial")
    if partial_output_path.exist?
      start = partial_output_path.size
      headers["Range"] = "bytes=#{start}-"
    end

    url = @url
    http = Net::HTTP.new(url.hostname, url.port)
    # http.set_debug_output($stderr)
    http.use_ssl = (url.scheme == "https")
    path = url.path
    path += "?#{url.query}" if url.query
    request = Net::HTTP::Get.new(path, headers)
    response = Gem::Request.request(url, request)

    if response.is_a?(Net::HTTPPartialContent)
      mode = "ab"
    else
      start = nil
      mode = "wb"
    end

    base_name = @url.path.split("/").last
    size_current = 0
    size_max = response.content_length
    if start
      size_current += start
      size_max += start
    end
    partial_output_path.open(mode) do |output|
      output.write(response.body)
    end
    FileUtils.mv(partial_output_path, output_path)
  rescue TooManyRedirects => error
    last_url = error.message[/\Atoo many redirections: (.+)\z/, 1]
    raise TooManyRedirects, "too many redirections: #{@url} .. #{last_url}"
  end

  def start_http(url, headers, limit = 10, &block)
    if limit == 0
      raise TooManyRedirects, "too many redirections: #{url}"
    end
    puts url.class, url, url.hostname
    http = Net::HTTP.new(url.hostname, url.port)
    # http.set_debug_output($stderr)
    http.use_ssl = (url.scheme == "https")
    path = url.path
    path += "?#{url.query}" if url.query
    request = Net::HTTP::Get.new(path, headers)
    response = Gem::Request.request(url, request)
    block.call(response)
  end
end
nil

In [9]:
require "charty"
require "datasets-daru"

Charty::Backends.use(:plotly)
charty = Charty::Plotter.new(:plotly)

iris = Datasets::Iris.new.to_daru

#<Daru::DataFrame(150x5)>

In [10]:
plot = charty.scatter do
  iris.group_by(:label).groups.each do |label, index|
    records = iris.row[*index]
    xs = records[:petal_length].to_a
    ys = records[:petal_width].to_a
    series xs, ys, label: label[0]
  end
  xlabel "Petal Length"
  ylabel "Petal Width"
  range x: 0..7, y: 0..3
end

#<Charty::RenderContext:0x2ebdf320>

In [11]:
content = JS.eval("return {}")
html = plot.render
content[:data] = JS.eval("return {}")
content[:data]["text/html"] = html
$JUPYTER_KERNEL.displayData(content)

undefined